In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from copy import deepcopy
from torch.nn.utils.rnn import pad_sequence
from utils.reparam_module import ReparamModule
import numpy as np
import random
from tqdm import tqdm

In [15]:
dataset_name = 'yelp-small'
full_dataset_name = 'yelp-small'
num_item_dict = {
    'toy': 11925,
    'sport': 18358,
    'beauty':12102,
    'yelp-small': 20034,
}
num_item = num_item_dict[dataset_name]
SOS = num_item
EOS = num_item + 1

In [16]:
interval = list(range(0, 40001, 10000))
rst_list = []
for i in range(len(interval) - 1):
    rst_list.append(torch.load(f'f-seq-con2-{dataset_name}-{interval[i]}-{interval[i + 1]}.pth'))
rst = []
for _ in rst_list:
    rst += _
print(len(rst))

152155


In [17]:
ori_pattern = torch.load(f'./dataset/{full_dataset_name}-noise-50/{dataset_name}/train_new-pure.pth')
ori_seqlist = [list(_[1][:_[3]]) + [_[2][_[3] - 1]] for _ in list(ori_pattern)]
len(ori_pattern)

68819

In [18]:
max_seq_len = 50
def truncate_or_pad(seq):
    cur_seq_len = len(seq)
    if cur_seq_len > max_seq_len:
        return seq[-max_seq_len:]
    else:
        return seq + [0] * (max_seq_len - cur_seq_len)

train_set = set()
# for _ in ori_seqlist:
#     train_set.add(tuple(_))

for pattern in rst:
    seq = pattern.tolist()[1:-1]
    train_set.add(tuple(seq))
print(len(train_set))

27437


In [19]:
train_list = []
for _ in train_set:
    seq_len = sum([a != 0 for a in list(_)[:-1]])
    if seq_len == 0:
        continue
    train_list.append([
        1,
        truncate_or_pad(list(_)[:-1]),
        truncate_or_pad(list(_)[1:]),
        seq_len,
        [1] * max_seq_len,
        [0] * max_seq_len,
    ])
print(len(train_list))

27237


In [20]:
train_ori = torch.load(f'./dataset/{full_dataset_name}-noise-50/{dataset_name}/train_ori.pth')

In [21]:
torch.save(train_list + ori_pattern + train_ori, f'./dataset/{full_dataset_name}-noise-50/{dataset_name}/train_gene-real2.pth')